In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"# this is potentially unsafe but who cares cos it gets rid of an error!

%cd "C:/Users/Ibrahim/biomechanics/yolov7"

filename= "CL_0.6_27"
file= str(filename) + ".mp4" 



!python detect_or_track.py --weights yolov7_custom.pt --source {file} --save-txt --device cpu --name {filename}

C:\Users\Ibrahim\biomechanics\yolov7
Namespace(weights=['yolov7_custom.pt'], source='CL_0.6_27.mp4', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='CL_0.6_27', exist_ok=False, no_trace=False, track=False, show_track=False, show_fps=False, thickness=2, seed=1, nobbox=False, nolabel=False, unique_track_color=False, save_bbox_dim=False, save_with_object_id=False)
Fusing layers... 
IAuxDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

video 1/1 (1/238) C:\Users\Ibrahim\biomechanics\yolov7\CL_0.6_27.mp4: 1 ball, Done. (1081.3ms) Inference, (1.0ms) NMS
video 1/1 (2/238) C:\Users\Ibrahim\biomechanics\yolov7\CL_0.6_27.mp4: 1 ball, Done. (924.7ms) Inference, (1.0ms) NMS
video 1/1 (3/238) C:\Users\Ibrahim\biomechanics\yolov7\CL_0.6_27.mp4: 1 ball, Done. (988.2ms) Inference, (1.0ms) NMS
v

YOLOR  d1485a0 torch 1.13.1+cpu CPU

C:\Users\Ibrahim\AppData\Roaming\Python\Python39\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 500 layers, 110331736 parameters, 0 gradients, 143.5 GFLOPS


In [ ]:
import cv2
filepath = "C:/Users/Ibrahim/biomechanics/yolov7/runs/detect/" + filename
%cd filepath

def click_event(event, x, y, flags, params):
   if event == cv2.EVENT_LBUTTONDOWN:
    #print(f'Original Coordinates : ({x},{y})')
    global xy 
    global yy
    xy = x/frame.shape[1]
    yy = y/frame.shape[0]
    print(f'YOLO Coordinates : ({xy},{yy})')
    cv2.putText(frame, f'({x},{y})',(x,y),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    cv2.circle(frame, (x,y), 8, (255,0,0), -1)
   
    return xy , yy
    

videopath = file

videcap = cv2.VideoCapture(videopath)

cv2.namedWindow("1")
cv2.setMouseCallback("1",click_event)    
            
while(videcap.isOpened()):
    ret, frame = videcap.read()
    if ret == True:
        cv2.imshow('1', frame)   

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
cv2.destroyAllWindows()


In [ ]:
## biomechanics yolo
#!python detect_or_track.py --weights yolov7_custom.pt --source CL_1_S0003.mp4 --save-txt --name CL_1_S0003
import numpy as np
import os
import pandas as pd
import scipy
import math


# Specify the folder containing the text files
folder_path = "./labels"

# Create an empty list to store the data from each file
all_data = []

# Iterate through all the files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        # Open the file and read its contents
        with open(os.path.join(folder_path, filename), 'r') as file:
            file_data = file.readline()

        # Split the line into a list of numbers
        line_data = file_data.strip().split(" ")

        # Convert the numbers to integers
        line_data = [float(x) for x in line_data]

        # Add the line data to the list of all data
        all_data.append(line_data)

# Convert the list of data to a pandas DataFrame
df = pd.DataFrame(all_data)
display(df)

df.to_csv('filename.csv', index=False)

# Write the DataFrame to a CSV file in the same folder as the text files
df.to_csv(os.path.join(folder_path, "concatenated_data.csv"), index=False)


# Set the directory to the current directory #os.chdir("C:\\Users\\Ibrahim\\desktop")
file= videopath #for name currently


x_wall = xy #coordinates of the wall, uses the global variables from the other script
y_wall = yy

num_cont_frames=0

#video = cv.VideoCapture(path)
ball_size = 0.22  #diameter of a regulation ball in meters
fps_cam = 10000  # Change this to the required fps of the video

# Initialize variable to track state of ball (in contact with wall or not)
in_contact = False

# Initialize variable to track whether in_contact has ever been True
in_contact_ever = False


# Initialize lists for inbound and outbound velocities
inbound_velocities = []
outbound_velocities = []

# Calculate time interval between frames in seconds
time_interval = 1 / fps_cam

scale  = []
x_def = []
inbound_x = []
inbound_y = []
outbound_x = []
outbound_y = []


set_size = 5 # adjust this value based on your data
threshold = 3 # adjust this value based on your data


for index, row in df.iterrows(): #change this to for each line of the pandas df
#x is x center y is y center
    x, y, w, h = row[1], row[2], row[3], row[4]
    x2=x+w/2
    y2=y+h/2
    x1=x-w/2
    y1=y-h/2
    
    scale.append(ball_size/w)   #meters per pixel.diameter in pixels or coordinate value / real diameter in m to give pixel per m for a scale factor



    start = max(0, i - set_size)
    end = min(len(df), i + set_size)
    x2_set = df.iloc[start:end][1] + df.iloc[start:end][3]/2
    y2_set = df.iloc[start:end][2] + df.iloc[start:end][4]/2
    x2_mean = x2_set.mean()
    y2_mean = y2_set.mean()
    x2_std = x2_set.std()
    y2_std = y2_set.std()
    if abs(x2 - x2_mean) > threshold * x2_std or abs(y2 - y2_mean) > threshold * y2_std:
        df.drop(index, inplace=True)
        continue
    # the rest of the code

    
    if x1 < x_wall: #sometimes the bbox is the wrong way around
        # Set in_contact to True
        in_contact = True
        # Set in_contact_ever to True
        in_contact_ever = True
        # Increment counter
        num_cont_frames = num_cont_frames + 1
        x_defe = x2-x_wall
        x_def.append(x_defe)
    else:
        in_contact = False

    if in_contact == False and in_contact_ever==False:
        inbound_x.append(x2) #list of x positions of right edge
        inbound_y.append(y2)

    if in_contact == False and in_contact_ever==True:
        outbound_x.append(x2) #list of x positions of right edge
        outbound_y.append(y2)

 
scale_ave=scipy.stats.trim_mean(scale, 0.3) #trim_mean 30% either way to remove some extraneous results


contact_time = num_cont_frames/fps_cam
print(contact_time)

if x_def:
    realxdef = min(x_def)*scale_ave
else:
    realxdef = 0

print(realxdef)

# Calculate inbound velocities
inbound_x_diff = []
inbound_y_diff = []
# Calculate inbound x- and y-velocities
inbound_x_velocities = []
inbound_y_velocities = []
inbound_len = len(inbound_x) - 1
print(inbound_x)

# Calculate differences between consecutive x and y coordinates
for i in range(inbound_len):
    inbound_x_diff.append(inbound_x[i] - inbound_x[i + 1])
    inbound_y_diff.append(inbound_y[i] - inbound_y[i + 1])
print(inbound_x_diff)

filtered_inbound_x_diff = []
filtered_inbound_y_diff = []

for i in range(inbound_len):
    if inbound_x_diff[i] <= 1:
        filtered_inbound_x_diff.append(inbound_x_diff[i])
inbound_x_diff = filtered_inbound_x_diff

for i in range(inbound_len):
    if inbound_y_diff[i] <= 1:
        filtered_inbound_y_diff.append(inbound_y_diff[i])
inbound_y_diff = filtered_inbound_y_diff

inbound_y_diff = [abs(value) for value in inbound_y_diff]
inbound_x_diff = [abs(value) for value in inbound_x_diff]

# Calculate inbound velocities in meters per second
inbound_velocities = []
for i in range(inbound_len):
    inbound_x_velocity = inbound_x_diff[i] * scale_ave * fps_cam
    inbound_x_velocities.append(inbound_x_velocity)
    inbound_y_velocity = inbound_y_diff[i] * scale_ave * fps_cam
    inbound_y_velocities.append(inbound_y_velocity)

# Calculate outbound velocities
outbound_x_diff = []
outbound_y_diff = []

outbound_len = len(outbound_x) - 1

# Calculate differences between consecutive x and y coordinates
for i in range(outbound_len):
    outbound_x_diff.append(outbound_x[i] - outbound_x[i + 1])
    outbound_y_diff.append(outbound_y[i] - outbound_y[i + 1])
    
outbound_y_diff = [abs(value) for value in outbound_y_diff]
outbound_x_diff = [abs(value) for value in outbound_x_diff]


    
filtered_outbound_x_diff = []
filtered_outbound_y_diff = []

for i in range(outbound_len):
    if outbound_x_diff[i] <= 1:
        filtered_outbound_x_diff.append(outbound_x_diff[i])
outbound_x_diff = filtered_outbound_x_diff

for i in range(outbound_len):
    if outbound_y_diff[i] <= 1:
        filtered_outbound_y_diff.append(outbound_y_diff[i])
outbound_y_diff = filtered_outbound_y_diff


# Calculate outbound velocities in meters per second
outbound_velocities = []
outbound_x_velocities = []
outbound_y_velocities = []

for i in range(outbound_len):
    outbound_x_velocity = outbound_x_diff[i] * scale_ave * fps_cam
    outbound_x_velocities.append(outbound_x_velocity)
    outbound_y_velocity = outbound_y_diff[i] * scale_ave * fps_cam
    outbound_y_velocities.append(outbound_y_velocity)

print(inbound_x_velocities)
  
    
filtered_outbound_x_velocities = []
for value in outbound_x_velocities:
    if value < 100:
        filtered_outbound_x_velocities.append(value)
outbound_x_velocities = filtered_outbound_x_velocities

filtered_outbound_y_velocities = []
for value in outbound_y_velocities:
    if value < 100:
        filtered_outbound_y_velocities.append(value)
outbound_y_velocities = filtered_outbound_y_velocities

filtered_inbound_x_velocities = []
for value in inbound_x_velocities:
    if value < 100:
        filtered_inbound_x_velocities.append(value)
inbound_x_velocities = filtered_inbound_x_velocities

filtered_inbound_y_velocities = []
for value in inbound_y_velocities:
    if value < 100:
        filtered_inbound_y_velocities.append(value)
inbound_y_velocities = filtered_inbound_y_velocities


corrected_average_inbound_x_velocities = scipy.stats.trim_mean(inbound_x_velocities, 0.2)
corrected_average_inbound_y_velocities = scipy.stats.trim_mean(inbound_y_velocities, 0.2)
corrected_average_inbound_velocities = math.hypot(corrected_average_inbound_y_velocities, corrected_average_inbound_x_velocities)
corrected_average_outbound_x_velocities = scipy.stats.trim_mean(outbound_x_velocities, 0.2)
corrected_average_outbound_y_velocities = scipy.stats.trim_mean(outbound_y_velocities, 0.2)
corrected_average_outbound_velocities = math.hypot(corrected_average_outbound_y_velocities, corrected_average_outbound_x_velocities)


# Create a dictionary with the data for the table
speeddata={'inbound x velocity ': corrected_average_inbound_x_velocities, 'inbound y velocity': corrected_average_inbound_y_velocities, 'inbound velocity': corrected_average_inbound_velocities, 'outbound x velocity': corrected_average_outbound_x_velocities, 'outbound y velocity': corrected_average_outbound_y_velocities, 'outbound velocity': corrected_average_outbound_velocities, 'contact time': contact_time, 'deformation' :realxdef}

df2 = pd.DataFrame(speeddata, index=range(len(speeddata)))

# Export the DataFrame to a CSV file
filename = file + 'results.csv'
df2.to_csv(filename, index=False)